# Исследование надёжности заёмщиков


In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Пропуски обнаружены в столбцах days_employed и total_income.В столбце days_employed помимо пропусков так же стоят минусы перед некоторыми значениями.В столбце education строки написаны в разных регистрах.Пропуски могли возникнуть из-за того что у человека не записан стаж в трудовой книге,по той же причине нет сведений о доходах(Мое предположение).

## Предобработка данных

### Обработка пропусков

In [3]:
print('Количество пропусков в столбце days_employed: ',df['days_employed'].isna().sum())
print('Количество пропусков в столбце total_income: ',df['total_income'].isna().sum())

Количество пропусков в столбце days_employed:  2174
Количество пропусков в столбце total_income:  2174


In [4]:
df['days_employed'] = df['days_employed'].fillna(0)
df['total_income'] = df['total_income'].fillna(0)
#теперь я хочу написать функцию которая принимает в себя столбец и убирает минусы
def magic(row):
    if row < 0:
        return row * -1
    else:
        return row
    
    
#вызовем нашу функцию и уберем минусы

df['days_employed'] = df['days_employed'].apply(magic)
#Исправим написание значений в столбце education
df['education'] = df['education'].str.lower()
#В столбце total_income вместо пропущенных значений поставим медианное значение зарплаты
#для этого посмотрим медианное значение по всем группам


In [5]:
df.groupby('income_type')['total_income'].median()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        139034.452386
компаньон          162401.351555
пенсионер          110179.690761
предприниматель    249581.572474
сотрудник          133546.457238
студент             98201.625314
Name: total_income, dtype: float64

In [6]:

#напишем функцию которая заменит 0 в столбце total_income на медианное значение в зависимости от рода деятельности респондента

def income_type_total_income(row):
    
    work = row['income_type']
    total = row['total_income']
    
    if work == 'безработный':
        if total == 0:
            return 131339.751676
    if work == 'в декрете':
        if total == 0:
            return 53829.130729
    if work == 'госслужащий':
        if total == 0:
            return 139304.472779
    if work == 'компаньон':
        if total == 0:
            return 162489.341762
    if work == 'пенсионер':
        if total == 0:
            return 110836.481000
    if work == 'предприниматель':
        if total == 0:
            return 249581.572474
    if work == 'сотрудник':
        if total == 0:
            return 133890.827873
    if work == 'студент':
        if total == 0:
            return 98201.625314
    else:
        return total
df['total_income'] = df.apply(income_type_total_income, axis = 1)


In [7]:
df['dob_years'].unique().mean()

46.189655172413794

In [8]:
#Средний возраст всех людей в данной таблице 46 лет ,считаю целесообразным в столбце days_employed
#вместо пропущенных значений поставить среднее значение рабочих дней
df['days_employed'] = df['days_employed'].replace(0,df['days_employed'].mean())
df.head(20)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21524 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

В столбце days_employed исчезли минусы и пропуски.Вместо пропусков поставлены средние значения.В столбце total_income вместо пропусков стоят медианные значения.В столбце education все значения приведены к единому регистру


### Замена типа данных

In [10]:
#в столбце days_employed переведу дни в годы
df['days_employed'] = df['days_employed'] / 365
#теперь переведу столбец days_employed в тип данных int
df['days_employed'] = df['days_employed'].astype(int)
#в столбце debt изменю тип данных на bool
df['debt'] = df['debt'].astype(bool)
#в столбце purpose и family_status изменю тип данных на str
df['purpose'] = df['purpose'].astype(str)
df['family_status'] = df['family_status'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null bool
total_income        21524 non-null float64
purpose             21525 non-null object
dtypes: bool(1), float64(1), int64(5), object(5)
memory usage: 1.8+ MB


In [11]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,False,253875.639453,покупка жилья
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,False,112080.014102,приобретение автомобиля
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,False,145885.952297,покупка жилья
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,False,267628.550329,дополнительное образование
4,0,932,53,среднее,1,гражданский брак,1,F,пенсионер,False,158616.077870,сыграть свадьбу


**Вывод**

В столбце days_employed наблюдаются неверные или искаженные значения так как врядли человек мог работать 932 года.Я использую метод astype() для изменения типов данных,так как он тут самый уместный.

### Обработка дубликатов

In [12]:
df.duplicated().sum()

71

In [13]:
df = df.drop_duplicates().reset_index(drop = True)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null bool
total_income        21453 non-null float64
purpose             21454 non-null object
dtypes: bool(1), float64(1), int64(5), object(5)
memory usage: 1.8+ MB


**Вывод**

Я обнаружил 71 строку с дубликатами и удалил их.Использовал при этом метод drop_duplicates.Дубликаты могли появится в следствии сбоя при выгрузке данных

### Лемматизация

In [15]:
from pymystem3 import Mystem
m = Mystem()
#хочу посмотреть на разнообразие целей кредита и заодно выясним самую популярную
df['purpose'].value_counts(sort = True)

#лемматизируем столбец purpose
df['purpose_lemmatize'] = df['purpose'].apply(m.lemmatize)

In [16]:
df['purpose_lemmatize'].value_counts()

[автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       768
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            624
[покупка

**Вывод**

при лемматизации столбца 'purpose' можем выделить группы для целей кредита:
1.жилье
2.коммерческая недвижимость
3.свадьба
4.образование
5.автомобиль

### Категоризация данных

In [17]:
#для категоризации данных напишу функцию которая будет принимать в себя данные столбца 'purpose_lemmatize' и возвращать 
#категорию из целей кредита
def category(row):
    for i in row:
        if i =='свадьба':
            return 'свадьба'
            
        elif i == 'автомобиль':
            return 'автомобиль'
        elif i == 'коммерческий':
            return 'коммерческая недвижимость'
        elif i == 'жилье' or i == 'недвижимость':
            return 'жилье'
        elif i == 'образование':
            return 'образование'
        
        
    
    
df['purpose_category'] = df['purpose_lemmatize'].apply(category)
df['purpose_category'].value_counts()  

жилье                        9500
автомобиль                   4306
образование                  4013
свадьба                      2324
коммерческая недвижимость    1311
Name: purpose_category, dtype: int64

**Вывод**

Больше всего кредитов люди берут на покупку собственного жилья

## Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [18]:
df['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В данных есть странное значение '-1'.Я не думаю что нам хотят сказать что ребенок был а потом его не стало.Скорее всего это ошибка в данных и ее нужно рассматривать как наличие ребенка.Так же есть значение '20' ,я тоже посчитаю это выбросом и учту как наличие детей.
Для удобства я собираюсь создать новый столбец children_category в котором категоризирую данные и в котором вместо колличества детей будет указано их наличие или отсутсвие.

In [19]:
def family(row):
    if row == 0:
        return 'детей нет'
    else:
        return 'дети есть'
    
df['children_category'] = df['children'].apply(family)
df['children_category'].value_counts()
    
    

детей нет    14091
дети есть     7363
Name: children_category, dtype: int64

In [20]:
df.groupby('children_category')['debt'].value_counts()

children_category  debt 
детей нет          False    13028
                   True      1063
дети есть          False     6685
                   True       678
Name: debt, dtype: int64

In [21]:
childfree = 1063/14091
withkids = 678/7363
print('Доля людей не имеющих детей у которых есть задолженность перед банком: {:.2%}'.format(childfree))
print('Доля людей у которых есть дети имеющих задолженность перед банком: {:.2%}'.format(withkids))

Доля людей не имеющих детей у которых есть задолженность перед банком: 7.54%
Доля людей у которых есть дети имеющих задолженность перед банком: 9.21%


**Вывод**

Люди имеющие детей чаще имеют задолженность по кредитам.Скорее всего это вызвано большими расходами на содержание детей

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [22]:
df.groupby('debt')['family_status'].value_counts()

debt   family_status        
False  женат / замужем          11408
       гражданский брак          3763
       Не женат / не замужем     2536
       в разводе                 1110
       вдовец / вдова             896
True   женат / замужем            931
       гражданский брак           388
       Не женат / не замужем      274
       в разводе                   85
       вдовец / вдова              63
Name: family_status, dtype: int64

Категоризирую данные столбца family_status по принципу в отношениях / одиночки

In [23]:
def status(row):
    if row == 'Не женат / не замужем' or row == 'в разводе' or row == 'вдовец / вдова':
        return 'одинок / одинока'
    else:
        return 'в отношениях'
    
df['family_status_category'] = df['family_status'].apply(status)
df.groupby('debt')['family_status_category'].value_counts()

debt   family_status_category
False  в отношениях              15171
       одинок / одинока           4542
True   в отношениях               1319
       одинок / одинока            422
Name: family_status_category, dtype: int64

In [24]:
single = 422/4964
couple = 1319/16490
print('Доля одиноких людей имеющих задолженность: {:.2%}'.format(single))
print('Доля людей состоящих в отношениях имеющих задолженность: {:.2%}'.format(couple))

Доля одиноких людей имеющих задолженность: 8.50%
Доля людей состоящих в отношениях имеющих задолженность: 8.00%


**Вывод**

Зависимость между семейным положением и возвратом кредита в срок не обнаружена

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

### Я собираюсь категоризировать данные стобца total_income о доходе по принципу:
#### 1.низкий доход до 60000 рублей
#### 2.средний доход от 60000 до 121000 рублей
#### 3.высокий доход от 121000 рублей
##### Данные о уровне доходов Россиян и причеслинию их к разным классам я взял из исследования аналитиков АКРА размещенном на сайте РБК

In [25]:
def money(row):
    if row <= 60000:
        return 'низкий доход'
    elif row > 60000 and row <= 121000:
        return 'средний доход'
    else:
        return 'высокий доход'
    
df['total_income_category'] = df['total_income'].apply(money)
df['total_income_category'].value_counts()

высокий доход    14122
средний доход     6526
низкий доход       806
Name: total_income_category, dtype: int64

In [26]:
df.groupby('debt')['total_income_category'].value_counts()

debt   total_income_category
False  высокий доход            12979
       средний доход             5977
       низкий доход               757
True   высокий доход             1143
       средний доход              549
       низкий доход                49
Name: total_income_category, dtype: int64

In [27]:
low = 49/806
medium = 514/6526
high = 1178/14122
print('Доля людей с низким доходом имеющих задолженность перед банком: {:.2%}'.format(low))
print('Доля людей со средним доходом имеющих задолженность перед банком: {:.2%}'.format(medium))
print('Доля людей с высоким доходом имеющих задолженность перед банком: {:.2%}'.format(high))

Доля людей с низким доходом имеющих задолженность перед банком: 6.08%
Доля людей со средним доходом имеющих задолженность перед банком: 7.88%
Доля людей с высоким доходом имеющих задолженность перед банком: 8.34%


**Вывод**

Зависимость установлена.Чем выше доход тем более высокая вероятность задолженности по кредиту

- Как разные цели кредита влияют на его возврат в срок?

In [28]:
df.groupby('debt')['purpose_category'].value_counts()

debt   purpose_category         
False  жилье                        8817
       автомобиль                   3903
       образование                  3643
       свадьба                      2138
       коммерческая недвижимость    1212
True   жилье                         683
       автомобиль                    403
       образование                   370
       свадьба                       186
       коммерческая недвижимость      99
Name: purpose_category, dtype: int64

In [29]:
house = 683/9500
car = 403/4306
education = 370/4013
wedding = 186/2324
buisness = 99/1311
print('Доля людей с целью кредита жилье имеющих задолженность: {:.2%}'.format(house))
print('Доля людей с целью кредита автомобиль имеющих задолженность: {:.2%}'.format(car))
print('Доля людей с целью кредита образование имеющих задолженность: {:.2%}'.format(education))
print('Доля людей с целью кредита свадьба имеющих задолженность: {:.2%}'.format(wedding))
print('Доля людей с целью кредита коммерческая недвижимость имеющих задолженность: {:.2%}'.format(buisness))

Доля людей с целью кредита жилье имеющих задолженность: 7.19%
Доля людей с целью кредита автомобиль имеющих задолженность: 9.36%
Доля людей с целью кредита образование имеющих задолженность: 9.22%
Доля людей с целью кредита свадьба имеющих задолженность: 8.00%
Доля людей с целью кредита коммерческая недвижимость имеющих задолженность: 7.55%


**Вывод**

Хуже всего кредит возвращают люди берущие деньги для образования или для покупки авто.Лучше всего отдают люди которые берут деньги покупки собственного жилья и недвижимости для бизнеса.Люди берущие деньги деньги для свадьбы имеют средний показатель в данной выборке.На мой взгляд зависимость установлена,ипотека является самым надежным займом для банков.

## Шаг 4. Общий вывод

На мой взгляд самая очевидная зависимость просматривается в целях кредита.Ипотека как я писал выше является зачастую очень взвешенным и обдуманным шагом для большинства людей.Покупка же авто в кредит на мой взгляд я вляется импульсивной покупкой,о чем свидетельствуют данные из таблицы выше.Признаюсь честно,ожидал что именно кредит за свадьбу будут отдавать хуже всего,но как мы видим я был не прав.Плохие показатели по кредитам на образование я связываю с тяжелым поиском работы после и во время учебы.

Я не обнаружил зависимостей между семейным положением и наличием детей и выплатой кредита в срок.
Меня удивили данные о том что люди со средним и высоким достатком хуже всех отдают кредит.На мой взгляд это тема для отдельного исследования,так как логичных и очевидных причин которые лежат на поверхности я не вижу.